In [2]:
import org.archive.archivespark._
import org.archive.archivespark.functions._
import org.archive.archivespark.specific.warc._
import java.io._

val warc_prefix=sys.env("WARC_FILENAME")
val warcfile="/data/" + warc_prefix + ".warc.gz"
val cdxPath ="/results/" + warc_prefix
val records = ArchiveSpark.load(WarcSpec.fromFiles(warcfile))

var summary = ""

var t1 = System.nanoTime
records.saveAsCdx(cdxPath + "/cdx.gz")
var duration = (System.nanoTime - t1) / 1e9d
summary = "CDX:" + duration + "|"

t1 = System.nanoTime
val pages = records.filter(r => r.mime == "text/html" && r.status == 200)
val Links = Html.all("a")
val LinkUrls = SURT.of(HtmlAttribute("href").ofEach(Links))
val LinkTexts = HtmlText.ofEach(Links)
val pagesWithLinks = pages.enrich(LinkUrls).enrich(LinkTexts)
pagesWithLinks.saveAsJson(cdxPath + "/pages-with-links.gz")
duration = (System.nanoTime - t1) / 1e9d
summary = summary + "PagesWithLinks:" + duration + "\n"

val pw = new PrintWriter(new File(cdxPath + "/"+ warc_prefix + "_results.txt" ))
pw.write(summary)
pw.close
